In [2]:
pip install kubeflow-training

  Using cached kubeflow_training-1.5.0-py3-none-any.whl (117 kB)
  Using cached kubernetes-24.2.0-py2.py3-none-any.whl (1.5 MB)
     |████████████████████████████████| 140 kB 2.3 MB/s            
     |████████████████████████████████| 161 kB 10.1 MB/s            
  Using cached retrying-1.3.3.tar.gz (10 kB)
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 173 kB 15.1 MB/s            
     |████████████████████████████████| 55 kB 12.7 MB/s            
     |████████████████████████████████| 62 kB 8.3 MB/s             
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached google_auth-2.12.0-py2.py3-none-any.whl (169 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached rsa-4.9-py3-none-any.whl (34 kB)
     |████████████████████████████████| 61 kB 835 kB/s             
  Using cached oauthlib-3.2.1-py3-none-any.whl (151 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
Using legacy 'set

In [3]:
from kubeflow import training 

In [4]:
pip install setuptools

You should consider upgrading via the '/usr/local/bin/python3.10 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [17]:
%ls
%cd training-operator/sdk/python/
%ls
%run setup.py install

OWNERS     README.md  docs/      examples/  kubeflow/  setup.py   test/
[Errno 2] No such file or directory: 'training-operator/sdk/python/'
/Users/erico07181/Documents/KubeflowPython/training-operator/sdk/python
OWNERS     README.md  docs/      examples/  kubeflow/  setup.py   test/
running install
running bdist_egg
running egg_info
creating kubeflow_training.egg-info
writing kubeflow_training.egg-info/PKG-INFO
writing dependency_links to kubeflow_training.egg-info/dependency_links.txt
writing requirements to kubeflow_training.egg-info/requires.txt
writing top-level names to kubeflow_training.egg-info/top_level.txt
writing manifest file 'kubeflow_training.egg-info/SOURCES.txt'
reading manifest file 'kubeflow_training.egg-info/SOURCES.txt'
writing manifest file 'kubeflow_training.egg-info/SOURCES.txt'
installing library code to build/bdist.macosx-10.9-universal2/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/test
copying test/test_kubeflow

In [18]:
%ls

OWNERS                      examples/
README.md                   kubeflow/
build/                      kubeflow_training.egg-info/
dist/                       setup.py
docs/                       test/


In [20]:


from kubernetes.client import V1PodTemplateSpec
from kubernetes.client import V1ObjectMeta
from kubernetes.client import V1PodSpec
from kubernetes.client import V1Container

from kubeflow.training import constants
from kubeflow.training.utils import utils
from kubeflow.training import V1ReplicaSpec
from kubeflow.training import KubeflowOrgV1TFJob
from kubeflow.training import KubeflowOrgV1TFJobSpec
from kubeflow.training import V1RunPolicy
from kubeflow.training import TFJobClient



In [21]:
namespace = utils.get_default_target_namespace()

THIS IS TO DEFINE THE TFJOB

In [22]:
container = V1Container(
    name="tensorflow",
    image="gcr.io/kubeflow-ci/tf-mnist-with-summaries:1.0",
    command=[
        "python",
        "/var/tf_mnist/mnist_with_summaries.py",
        "--log_dir=/train/logs", "--learning_rate=0.01",
        "--batch_size=150"
        ]
)

worker = V1ReplicaSpec(
    replicas=2,
    restart_policy="Never",
    template=V1PodTemplateSpec(
        spec=V1PodSpec(
            containers=[container]
        )
    )
)

chief = V1ReplicaSpec(
    replicas=1,
    restart_policy="Never",
    template=V1PodTemplateSpec(
        spec=V1PodSpec(
            containers=[container]
        )
    )
)

ps = V1ReplicaSpec(
    replicas=1,
    restart_policy="Never",
    template=V1PodTemplateSpec(
        spec=V1PodSpec(
            containers=[container]
        )
    )
)

tfjob = KubeflowOrgV1TFJob(
    api_version="kubeflow.org/v1",
    kind="TFJob",
    metadata=V1ObjectMeta(name="mnist",namespace=namespace),
    spec=KubeflowOrgV1TFJobSpec(
        run_policy=V1RunPolicy(clean_pod_policy="None"),
        tf_replica_specs={"Worker": worker,
                          "Chief": chief,
                          "PS": ps}
    )
)

CREATE THE TFJOB

In [ ]:
tfjob_client = TFJobClient()
tfjob_client.create(tfjob, namespace=namespace)


FETCH THE TFJOB 

In [ ]:
tfjob_client.get('mnist', namespace=namespace)

GET AND CHECK TFJOB STATUS

In [ ]:
tfjob_client.get_job_status('mnist', namespace=namespace)

WAIT FOR JOB TO FINISH

In [ ]:
tfjob_client.wait_for_job('mnist', namespace=namespace, watch=True)

CHECK IF SUCCEECED

In [ ]:
tfjob_client.is_job_succeeded('mnist', namespace=namespace)

GET TRAINING LOGS

In [ ]:
tfjob_client.get_logs('mnist', namespace=namespace)

DELETE THE TFJOB

In [ ]:
tfjob_client.delete('mnist', namespace=namespace)